In [1]:
# MNIST 데이터 준비

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# CNN 모델 - 2차원 평면
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

#drop out 비율
keep_prob  = tf.placeholder(tf.float32)

In [3]:
# 계층 1 정의
# W1 [3 3 1 32] -> [3 3]: 커널 크기(가로, 세로), 1: 입력값 X 의 특성수(크기), 32: 필터 갯수(채널)
# L1 Conv shape=(?, 28, 28, 32)
# Pool ->(?, 14, 14, 32)

#Convolution
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding='SAME')

#Pooling
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides = [1,2,2,1], padding='SAME')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# 계층 2 정의
# L2 Conv shape=(?, 14, 14, 64)
# Pool ->(?, 7, 7, 64)

#Convolution
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)

#Pooling
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1,2,2,1], padding='SAME')

In [5]:
# 완전 연결 레이어: 입력값 7x7x64 -> 출력값 256
# 직전의 Pool 사이즈인 (?, 7, 7, 64) 를 참고하여 차원을 줄임
# Reshape ->(?, 256)

W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev = 0.01))
L3 = tf.reshape(L2, [-1,7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

L3 = tf.nn.dropout(L3, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
# 최종 출력값 L3 에서의 출력 256개를 입력값으로 받아서
# 0~9 레이블인 10개의 출력값을 만듭니다.

W4 = tf.Variable(tf.random_normal([256,10], stddev = 0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels=Y))
#optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost) #속도 더 빠름

In [7]:
#신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #이미지 데이터를 CNN 모델을 위한 자료 형태 [28 28 1]의 형태로 재구성
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y:batch_ys, keep_prob: 0.7})

        total_cost += cost_val

        print('Epoch:', '%04d' % (epoch+1),'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
answer = sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1,28,28,1), Y: mnist.test.labels, keep_prob: 1})

print('정확도: {:.4}%'.format(answer*100))

Epoch: 0001 Avg. cost = 0.004
Epoch: 0001 Avg. cost = 0.008
Epoch: 0001 Avg. cost = 0.013
Epoch: 0001 Avg. cost = 0.017
Epoch: 0001 Avg. cost = 0.021
Epoch: 0001 Avg. cost = 0.025
Epoch: 0001 Avg. cost = 0.029
Epoch: 0001 Avg. cost = 0.033
Epoch: 0001 Avg. cost = 0.038
Epoch: 0001 Avg. cost = 0.042
Epoch: 0001 Avg. cost = 0.046
Epoch: 0001 Avg. cost = 0.050
Epoch: 0001 Avg. cost = 0.054
Epoch: 0001 Avg. cost = 0.059
Epoch: 0001 Avg. cost = 0.063
Epoch: 0001 Avg. cost = 0.067
Epoch: 0001 Avg. cost = 0.071
Epoch: 0001 Avg. cost = 0.075
Epoch: 0001 Avg. cost = 0.080
Epoch: 0001 Avg. cost = 0.084
Epoch: 0001 Avg. cost = 0.088
Epoch: 0001 Avg. cost = 0.092
Epoch: 0001 Avg. cost = 0.096
Epoch: 0001 Avg. cost = 0.100
Epoch: 0001 Avg. cost = 0.105
Epoch: 0001 Avg. cost = 0.109
Epoch: 0001 Avg. cost = 0.113
Epoch: 0001 Avg. cost = 0.117
Epoch: 0001 Avg. cost = 0.121
Epoch: 0001 Avg. cost = 0.126
Epoch: 0001 Avg. cost = 0.130
Epoch: 0001 Avg. cost = 0.134
Epoch: 0001 Avg. cost = 0.138
Epoch: 000

Epoch: 0001 Avg. cost = 0.804
Epoch: 0001 Avg. cost = 0.805
Epoch: 0001 Avg. cost = 0.805
Epoch: 0001 Avg. cost = 0.806
Epoch: 0001 Avg. cost = 0.806
Epoch: 0001 Avg. cost = 0.807
Epoch: 0001 Avg. cost = 0.807
Epoch: 0001 Avg. cost = 0.807
Epoch: 0001 Avg. cost = 0.808
Epoch: 0001 Avg. cost = 0.809
Epoch: 0001 Avg. cost = 0.810
Epoch: 0001 Avg. cost = 0.810
Epoch: 0001 Avg. cost = 0.810
Epoch: 0001 Avg. cost = 0.811
Epoch: 0001 Avg. cost = 0.812
Epoch: 0001 Avg. cost = 0.812
Epoch: 0001 Avg. cost = 0.813
Epoch: 0001 Avg. cost = 0.813
Epoch: 0001 Avg. cost = 0.814
Epoch: 0001 Avg. cost = 0.814
Epoch: 0001 Avg. cost = 0.814
Epoch: 0001 Avg. cost = 0.815
Epoch: 0001 Avg. cost = 0.815
Epoch: 0001 Avg. cost = 0.816
Epoch: 0001 Avg. cost = 0.816
Epoch: 0001 Avg. cost = 0.816
Epoch: 0001 Avg. cost = 0.817
Epoch: 0001 Avg. cost = 0.818
Epoch: 0001 Avg. cost = 0.818
Epoch: 0001 Avg. cost = 0.818
Epoch: 0001 Avg. cost = 0.819
Epoch: 0001 Avg. cost = 0.819
Epoch: 0001 Avg. cost = 0.819
Epoch: 000

Epoch: 0001 Avg. cost = 0.891
Epoch: 0001 Avg. cost = 0.892
Epoch: 0002 Avg. cost = 0.000
Epoch: 0002 Avg. cost = 0.000
Epoch: 0002 Avg. cost = 0.001
Epoch: 0002 Avg. cost = 0.001
Epoch: 0002 Avg. cost = 0.001
Epoch: 0002 Avg. cost = 0.001
Epoch: 0002 Avg. cost = 0.001
Epoch: 0002 Avg. cost = 0.001
Epoch: 0002 Avg. cost = 0.002
Epoch: 0002 Avg. cost = 0.002
Epoch: 0002 Avg. cost = 0.002
Epoch: 0002 Avg. cost = 0.002
Epoch: 0002 Avg. cost = 0.003
Epoch: 0002 Avg. cost = 0.003
Epoch: 0002 Avg. cost = 0.003
Epoch: 0002 Avg. cost = 0.003
Epoch: 0002 Avg. cost = 0.004
Epoch: 0002 Avg. cost = 0.004
Epoch: 0002 Avg. cost = 0.004
Epoch: 0002 Avg. cost = 0.004
Epoch: 0002 Avg. cost = 0.004
Epoch: 0002 Avg. cost = 0.005
Epoch: 0002 Avg. cost = 0.005
Epoch: 0002 Avg. cost = 0.005
Epoch: 0002 Avg. cost = 0.005
Epoch: 0002 Avg. cost = 0.006
Epoch: 0002 Avg. cost = 0.006
Epoch: 0002 Avg. cost = 0.006
Epoch: 0002 Avg. cost = 0.006
Epoch: 0002 Avg. cost = 0.006
Epoch: 0002 Avg. cost = 0.007
Epoch: 000

Epoch: 0002 Avg. cost = 0.051
Epoch: 0002 Avg. cost = 0.051
Epoch: 0002 Avg. cost = 0.051
Epoch: 0002 Avg. cost = 0.051
Epoch: 0002 Avg. cost = 0.051
Epoch: 0002 Avg. cost = 0.051
Epoch: 0002 Avg. cost = 0.052
Epoch: 0002 Avg. cost = 0.052
Epoch: 0002 Avg. cost = 0.052
Epoch: 0002 Avg. cost = 0.052
Epoch: 0002 Avg. cost = 0.052
Epoch: 0002 Avg. cost = 0.052
Epoch: 0002 Avg. cost = 0.052
Epoch: 0002 Avg. cost = 0.053
Epoch: 0002 Avg. cost = 0.053
Epoch: 0002 Avg. cost = 0.053
Epoch: 0002 Avg. cost = 0.053
Epoch: 0002 Avg. cost = 0.053
Epoch: 0002 Avg. cost = 0.053
Epoch: 0002 Avg. cost = 0.053
Epoch: 0002 Avg. cost = 0.054
Epoch: 0002 Avg. cost = 0.054
Epoch: 0002 Avg. cost = 0.054
Epoch: 0002 Avg. cost = 0.054
Epoch: 0002 Avg. cost = 0.054
Epoch: 0002 Avg. cost = 0.054
Epoch: 0002 Avg. cost = 0.054
Epoch: 0002 Avg. cost = 0.055
Epoch: 0002 Avg. cost = 0.055
Epoch: 0002 Avg. cost = 0.055
Epoch: 0002 Avg. cost = 0.055
Epoch: 0002 Avg. cost = 0.055
Epoch: 0002 Avg. cost = 0.055
Epoch: 000

Epoch: 0002 Avg. cost = 0.090
Epoch: 0002 Avg. cost = 0.090
Epoch: 0002 Avg. cost = 0.091
Epoch: 0002 Avg. cost = 0.091
Epoch: 0003 Avg. cost = 0.000
Epoch: 0003 Avg. cost = 0.000
Epoch: 0003 Avg. cost = 0.000
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.001
Epoch: 0003 Avg. cost = 0.002
Epoch: 0003 Avg. cost = 0.002
Epoch: 0003 Avg. cost = 0.002
Epoch: 0003 Avg. cost = 0.002
Epoch: 0003 Avg. cost = 0.002
Epoch: 0003 Avg. cost = 0.002
Epoch: 0003 Avg. cost = 0.002
Epoch: 0003 Avg. cost = 0.003
Epoch: 0003 Avg. cost = 0.003
Epoch: 0003 Avg. cost = 0.003
Epoch: 0003 Avg. cost = 0.003
Epoch: 0003 Avg. cost = 0.003
Epoch: 0003 Avg. cost = 0.003
Epoch: 0003 Avg. cost = 0.003
Epoch: 000

...

Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.013
Epoch: 0013 Avg. cost = 0.014
Epoch: 0013 Avg. cost = 0.014
Epoch: 0013 Avg. cost = 0.014
Epoch: 0013 Avg. cost = 0.014
Epoch: 0013 Avg. cost = 0.014
Epoch: 0013 Avg. cost = 0.014
Epoch: 0014 Avg. cost = 0.000
Epoch: 0014 Avg. cost = 0.000
Epoch: 0014 Avg. cost = 0.000
Epoch: 0014 Avg. cost = 0.000
Epoch: 0014 Avg. cost = 0.000
Epoch: 0014 Avg. cost = 0.000
Epoch: 0014 Avg. cost = 0.000
Epoch: 001

Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 0014 Avg. cost = 0.005
Epoch: 001

Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.011
Epoch: 0014 Avg. cost = 0.012
Epoch: 0014 Avg. cost = 0.012
Epoch: 0014 Avg. cost = 0.012
Epoch: 0014 Avg. cost = 0.012
Epoch: 0014 Avg. cost = 0.012
Epoch: 0014 Avg. cost = 0.012
Epoch: 0014 Avg. cost = 0.012
Epoch: 0014 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.000
Epoch: 0015 Avg. cost = 0.000
Epoch: 0015 Avg. cost = 0.000
Epoch: 0015 Avg. cost = 0.000
Epoch: 0015 Avg. cost = 0.000
Epoch: 001

Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.005
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 0015 Avg. cost = 0.006
Epoch: 001

Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
Epoch: 0015 Avg. cost = 0.012
최적화 완료
정확도: 99.19%
